In [1]:

# from tensorflow.keras.utils import np_utils
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import model_from_json
from tensorflow.keras.models import load_model 
from tensorflow.keras.constraints import max_norm
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import *
from tensorflow.keras.regularizers import l2

In [2]:
mirrored_strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy(
    tf.distribute.experimental.CollectiveCommunication.NCCL)
print(mirrored_strategy)

INFO:tensorflow:Using MirroredStrategy with devices ('/device:GPU:0',)
INFO:tensorflow:Single-worker MultiWorkerMirroredStrategy with local_devices = ('/device:GPU:0',), communication = CollectiveCommunication.NCCL


In [3]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [4]:
img_rows, img_cols= 28, 28
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

In [5]:
batch_size = 64
epochs = 1

In [6]:
def cnn(x_train, y_train, x_test, y_test, batch_size, epochs, input_shape):
    nb_classes= 10
    L_weight= 0.01
    
    model = Sequential()
    model.add( Conv2D(32, (3, 3), padding='valid', strides=1, 
                     kernel_constraint=max_norm(3), bias_constraint=max_norm(3),
                     bias_regularizer=l2(L_weight),
                     input_shape=input_shape, activation='relu'))
    
    model.add(Dropout(0.3))
    model.add(MaxPooling2D(pool_size=(1, 3), strides=(1, 3)))
    
    model.add( Conv2D(32, (3, 3), padding='valid', strides=1, 
                     kernel_constraint=max_norm(3), bias_constraint=max_norm(3),
                     bias_regularizer=l2(L_weight),
                     input_shape=input_shape, activation='relu'))
    model.add(Dropout(0.3))
    model.add(MaxPooling2D(pool_size=(1, 3), strides=(1, 3)))

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5)) 

    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))

    model.add(Dense(nb_classes))
    model.add(Activation('softmax'))

#     model.compile(loss='sparse_categorical_crossentropy',
#                   optimizer='adadelta',
#                   metrics=['accuracy'])
    
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

    model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])  
    
    
    history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs,
                        verbose=1, validation_data=(x_test, y_test))

    return model, history

In [7]:
with mirrored_strategy.scope():
    model, history= cnn(x_train, y_train, x_test, y_test, batch_size, epochs, input_shape)


Instructions for updating:
Use `tf.data.Iterator.get_next_as_optional()` instead.


UnknownError: 2 root error(s) found.
  (0) Unknown:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node sequential/conv2d/Conv2D (defined at /usr/lib/python3.6/threading.py:916) ]]
	 [[div_no_nan_1/Identity_1/ReadVariableOp/_18]]
  (1) Unknown:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node sequential/conv2d/Conv2D (defined at /usr/lib/python3.6/threading.py:916) ]]
0 successful operations.
0 derived errors ignored. [Op:__inference_train_function_1437]

Errors may have originated from an input operation.
Input Source operations connected to node sequential/conv2d/Conv2D:
 cond_1/Identity (defined at <ipython-input-6-10d848d31f17>:43)

Input Source operations connected to node sequential/conv2d/Conv2D:
 cond_1/Identity (defined at <ipython-input-6-10d848d31f

In [ ]:
print("done")